In [ ]:
%%capture
!pip install transformers

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard imoprt SummaryWriter
import numpy as np
import pandas as pd
from transformers import AutoTokenizer#, AutoModelForSequenceClassification, DistilBertForSequenceClassification, DistilBertTokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
#pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 1. Load Data

In [ ]:
def read_tweet_neutral_all(path):
  df = pd.read_csv(path, delimiter="\t", header=[0,1,2])
  df = pd.DataFrame(df.values.reshape(-1,3), columns=["tweet", "token", "label"])
  tweets = df.tweet.values
  tokens = df.token.values
  labels = df.token.values
  return tweets, tokens, labels



In [ ]:
tweets, tokens, labels = read_tweet_neutral_all("train.tsv")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
tweet_encodings = [tokenizer(tweet, truncation=True, padding=True) for tweet in tweets]
token_encodings = [tokenizer(token, truncation=True, padding=True) for token in tokens]

In [ ]:
class CryptoDataSet(T.utils.data.Dataset):
  def __init__(self, src_file, num_rows=None):
    x_load = np.loadtxt(srcfile, max_rows=num_rows,
                        usecols=0, delimiter",", #adjust usecols val
                        skiprows=0, dtype='U')
    y_load = np.loadtxt(src_file, max_rows=num_rows,
                        usecols=2, delimiter=",",skiprows=0,
                        dtype=np.long)
    
    # run through tokenizer here

    self.x_data = torch.tensor(x_load,
      dtype=T.).to(device)
    self.y_data = torch.tensor(y_load,
      dtype=T.long).to(device)

  def __len__(self):
    return len(self.x_data)

In [ ]:
sentence_pos = "I love the chair so much"
target_pos = "chair"
sentence_neg = "I hate the weather"
target_neg = "weather"

In [ ]:
inputs_pos, inputs_neg, target_input_pos, target_input_neg = tokenizer(sentence_pos), tokenizer(sentence_neg), tokenizer(target_pos), tokenizer(target_neg)

In [ ]:
type(inputs_pos)

transformers.tokenization_utils_base.BatchEncoding

In [ ]:
product = inputs_pos *  target_input_pos.torch.t


AttributeError: ignored

# 2. Implement Model

In [ ]:
PRE_TRAINED_MODEL_NAME = 'distilroberta-base'
BATCH_SIZE = 16
MAX_LEN = 160

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class_names = ['negative', 'neutral', 'positive']

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
model = SentimentClassifier(len(class_names))
model = model.to(device)

# 3.Train

In [ ]:
EPOCHS = 10

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
# training function
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples
):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
# function to evaluate model on given data loader
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)
      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
# training loop and storing training history
%%time
history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')
  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )
  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

In [ ]:
# plot training history
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);

In [ ]:
# download pretrained model on google play reviews here
# !gdown --id 1V8itWtowCYnb2Bc9KlK9SxGff9WwmogA
# model = SentimentClassifier(len(class_names))
# model.load_state_dict(torch.load('best_model_state.bin'))
# model = model.to(device)

# 4. Evaluation

In [ ]:
# calculating accuracy on test data
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)
test_acc.item()


In [ ]:
# helper function to get predictions from the model
def get_predictions(model, data_loader):

  model = model.eval()

  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():

    for d in data_loader:

      texts = d["review_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      
      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values